### Original example from: https://github.com/cajal/microns_phase3_nda/blob/v8/tutorial_notebooks/Matched_Cell_Functional_Data.ipynb

In [1]:
import os
# running from Root directory
os.chdir("..")

In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# from caveclient import CAVEclient
# import meshparty.skeleton_io

In [12]:
#client = CAVEclient('minnie65_public')
# client.materialize.get_tables()
# co_df = client.materialize.query_table('coregistration_auto_phase3_fwd_apl_vess_combined')
# co_df.to_csv('data/coregistration_auto_phase3_fwd_apl_vess_combined.csv')

co_df = pd.read_csv('data/coregistration_auto_phase3_fwd_apl_vess_combined.csv', index_col=0)
co_df.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
0,388873,2020-09-28 22:45:17.303349+00:00,t,357.495603,97818258123783846,864691135938031925,1,2024-05-23 16:26:39.793351+00:00,t,388873,4,7,107,1,18.1918,1.88120,[240496 103184 15068],[nan nan nan],[nan nan nan]
1,357184,2020-09-28 22:45:21.668338+00:00,t,398.268826,95566801907418600,864691135730121401,2,2024-05-23 16:26:39.793945+00:00,t,357184,4,7,127,1,13.0387,-7.90215,[223792 106080 15013],[nan nan nan],[nan nan nan]
2,357165,2020-09-28 22:44:28.479993+00:00,t,254.621450,94792402124187373,864691135737064068,3,2024-05-23 16:26:39.794546+00:00,t,357165,4,7,152,1,15.7229,3.58140,[218304 103136 15112],[nan nan nan],[nan nan nan]
3,290596,2020-09-28 22:41:22.519134+00:00,t,251.790459,89656239713364965,864691135338589414,4,2024-05-23 16:26:39.795104+00:00,t,290596,4,7,243,1,24.0639,3.89930,[181136 108880 15012],[nan nan nan],[nan nan nan]
4,578537,2020-09-28 22:41:28.675248+00:00,t,246.512845,112031301338636911,864691135291722934,5,2024-05-23 16:26:39.795684+00:00,t,578537,4,7,381,1,18.6186,5.59030,[343616 92608 15054],[nan nan nan],[nan nan nan]


In [13]:
neurons_df = pd.read_csv('data/connectome_neurons.csv',index_col=0)
neurons_df.head()

,root_id,volume,clf_type,cell_type,mtype,ds_num_of_pre_synapses,ds_num_of_post_synapses,ds_pre_syn_mean_weight,ds_post_syn_mean_weight,ds_pre_syn_sum_weight,...,num_of_inh_pre_synapses,pre_syn_mean_weight,pre_syn_sum_weight,pre_syn_std_weight,ex_pre_syn_mean_weight,ex_pre_syn_sum_weight,ex_pre_syn_std_weight,inh_pre_syn_mean_weight,inh_pre_syn_sum_weight,inh_pre_syn_std_weight
0,864691134784325628,234.475029,E,6P-IT,L6tall-c,50,3,4245.760000,3470.666667,212288,...,0,1096.000000,1096.0,0.000000,1096.000000,1096.0,0.000000,NaN,0.0,NaN
1,864691134807855033,313.159680,E,6P-IT,L6tall-c,68,4,4804.588235,1398.000000,326712,...,0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
2,864691134884743162,318.771200,E,6P-IT,L6short-a,1661,9,6338.362432,2671.111111,10528020,...,23,5991.680000,299584.0,5225.939232,6356.888889,171636.0,5416.703640,5562.956522,127948.0,4958.501908
3,864691134884747514,207.237120,E,6P-IT,L6short-b,659,28,7949.116844,4742.714286,5238468,...,5,4935.680000,123392.0,4389.639135,5514.800000,110296.0,4687.453569,2619.200000,13096.0,1322.185827
4,864691134884754426,205.180109,E,6P-CT,L6tall-a,1779,557,6489.351321,6157.572711,11544556,...,34,5751.284211,546372.0,5643.887961,5548.327869,338448.0,5937.430666,6115.411765,207924.0,5054.471021


In [14]:
neuron_w_activity_ids = set(co_df.pt_root_id)
print(len(neuron_w_activity_ids))

n_root_ids = set(neurons_df.root_id)
print(len(n_root_ids))

ids = neuron_w_activity_ids.intersection(n_root_ids) 
print(len(ids))

36505
71736
34726


### Sort post-synaptic neurons 
sort by how many pre-synaptic neurons has functional / acitivty data 

In [15]:
from connectome_download import load_neurons_full_dict
neurons_dict = load_neurons_full_dict()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71736/71736 [05:24<00:00, 220.89it/s]


In [31]:
from tqdm.notebook import trange, tqdm

neurons_w_pre_syn_activity = {}

for n in tqdm(neurons_dict.values()):
    pre_syn_neurons = [syn.pre_pt_root_id for syn in n.pre_synapses]
    pre_neuron_w_acitivty = sum(1 for pre_id in pre_syn_neurons if pre_id in neuron_w_activity_ids)
    neurons_w_pre_syn_activity[n.root_id] = pre_neuron_w_acitivty
    
sorted_dict = dict(sorted(neurons_w_pre_syn_activity.items(), key=lambda item: item[1], reverse=True))
sorted_dict

  0%|          | 0/71736 [00:00<?, ?it/s]

{864691135489514810: 1336,
 864691135397928737: 1204,
 864691135786756292: 1137,
 864691135884866160: 1098,
 864691136391836799: 1025,
 864691135273485073: 1016,
 864691135725334187: 965,
 864691135938397188: 964,
 864691135865240702: 961,
 864691136452249087: 959,
 864691135136336153: 955,
 864691135293026230: 927,
 864691135801046882: 921,
 864691135941021862: 912,
 864691135725448619: 910,
 864691135119058397: 908,
 864691136578814228: 904,
 864691136362878690: 899,
 864691135731577017: 896,
 864691134886151674: 895,
 864691135877536083: 891,
 864691135114472857: 881,
 864691136619729037: 868,
 864691135940617126: 860,
 864691135777597613: 849,
 864691136991202453: 844,
 864691136389585015: 839,
 864691136363235554: 838,
 864691136436882846: 837,
 864691135617065065: 836,
 864691135135829529: 835,
 864691136195466198: 830,
 864691136662432990: 829,
 864691136237733647: 828,
 864691136239073340: 824,
 864691135860270952: 821,
 864691134940999267: 819,
 864691135183273730: 817,
 86469

### Get functional data

In [4]:
# from microns_phase3 import nda, utils

In [40]:
# sample_neuron = list(ids)[0]
# entry = co_df[co_df.pt_root_id == sample_neuron].iloc[0]
# unit_key = entry[['session', 'scan_idx', 'unit_id']]
# dict(unit_key)